In [ ]:
# Import necessary libraries
from surprise import Dataset, Reader, KNNBasic
import pandas as pd
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Read Data

In [ ]:
# Read Data from Food_Data, Food_Rating, and Food_Tags
food_data = pd.read_csv("ORS DataV1.1/Food_Data.csv")

food_rating = pd.read_csv('ORS DataV1.1/Food_Rating.csv')

food_tags = pd.read_csv("ORS DataV1.1/food_tags.csv")

# Collaborative Filtering of Food Items based on User Ratings

In [ ]:
# Merge dataframes to create a unified dataset
merged_data = pd.merge(food_data, food_rating, on='FoodID')
merged_data = pd.merge(merged_data, food_tags, on=['FoodID', 'UserID'])

In [ ]:
merged_data

In [ ]:
# Define a reader to parse the data
reader = Reader(rating_scale=(1, 5))

In [ ]:
# Load data into Surprise dataset
data = Dataset.load_from_df(merged_data[['UserID', 'FoodID', 'Rating']], reader)


In [ ]:
data

In [ ]:
# Build a collaborative filtering model (k-NN algorithm)
sim_options = {
    'name': 'cosine',  # Use cosine similarity
    'user_based': False  # Item-based collaborative filtering
}
model = KNNBasic(sim_options=sim_options)


In [ ]:
# Train the model
trainset = data.build_full_trainset()
model.fit(trainset)

In [ ]:
# Function to get recommendations for a given food item
def get_recommendations_for_fooditem(food_name):
    # Find the FoodID for the given food_name
    food_id = food_data[food_data['FoodName'] == food_name]['FoodID'].values[0]

    # Get the inner ID for the food in the model
    food_inner_id = model.trainset.to_inner_iid(food_id)

    # Get the 5 nearest neighbors for the given food
    food_neighbors = model.get_neighbors(food_inner_id, k=5)

    # Convert inner IDs back to FoodIDs
    food_neighbors = [model.trainset.to_raw_iid(inner_id) for inner_id in food_neighbors]

    # Get the names of the recommended foods
    recommended_foods = food_data[food_data['FoodID'].isin(food_neighbors)]['FoodName'].tolist()

    return recommended_foods

In [ ]:
# Example: Get recommendations for Food Item
food_item = input("Please enter a food item to get recommendations")
recommendations = get_recommendations_for_fooditem(food_item)
print("FOOD - to - Food ORS running to fetch results..")

print(f"Recommended foods with {food_item} : {recommendations}")


# USER SIMILARITY Based on preference of users with similar taste

In [ ]:
# Merge dataframes to create a user-item matrix
user_item_matrix = pd.merge(food_rating, food_data, on="FoodID")

# Create a pivot table for user-item matrix
pivot_table = user_item_matrix.pivot_table(index='UserID', columns='FoodName', values='Rating', fill_value=0)

# Calculate cosine similarity between users
user_similarity = cosine_similarity(pivot_table.T)

In [ ]:
pivot_table

In [ ]:
# Recommend items based on user preferences
def recommend_food_on_UserID(user_id, n=5):
    user_index = pivot_table.index.get_loc(user_id)
    most_similar_user_index = max(range(len(user_similarity[user_index])), key=lambda i: user_similarity[user_index][i])

    user_data = user_item_matrix[user_item_matrix['UserID'] == most_similar_user_index + 1]
    top_rated_food = user_data.sort_values(by='Rating', ascending=False)['FoodName'].unique().tolist()

    # Filter out items the target user has already rated
    top_rated_food = [food for food in top_rated_food if food not in pivot_table.columns[user_index]]

    # Limit to the top n recommendations
    final_recommendations = top_rated_food[:n]
    print(f"Most Similar User {most_similar_user_index+1} Recommendations: {final_recommendations}")
    return final_recommendations


In [ ]:
# Example: Get recommendations for User
User_ID = int(input("Please enter a UserID to get recommendations"))
recommen = recommend_food_on_UserID(User_ID, n=3)
print("MOST SIMILAR USER ORS running to fetch results..")

print(f"Recommended foods for {User_ID} : {recommen}")

In [ ]:
# Recommend items based on user preferences
def recommend_food_on_n_similar_users(user_id, n=5, num_similar_users=5):
    user_index = pivot_table.index.get_loc(user_id)
    similar_users_indices = sorted(range(len(user_similarity[user_index])), key=lambda i: user_similarity[user_index][i], reverse=True)[1:num_similar_users + 1]

    common_items = set()

    for i in similar_users_indices:
        user_data = user_item_matrix[user_item_matrix['UserID'] == i + 1]
        top_rated_food = user_data.sort_values(by='Rating', ascending=False)['FoodName'].unique().tolist()

        # Filter out items the target user has already rated
        top_rated_food = [food for food in top_rated_food if food not in pivot_table.columns[user_index]]

        # Update the set with unique items from each similar user
        common_items.update(top_rated_food)

    # Limit to the top n common items
    final_common_items = list(common_items)[:n]
    print(f"Top {n} Common Items from Top {num_similar_users} Similar Users: {final_common_items}")
    return final_common_items


In [ ]:
# Example: Get recommendations for User
User_ID = int(input("Please enter a UserID to get recommendations"))
recommen = recommend_food_on_UserID(User_ID, n=3)
print("Top N SIMILAR USERS ORS running to fetch results..")

print(f"Recommended foods for {User_ID} : {recommen}")

# Most sold, frequently bought food items, Not taking Ratings into consideration

In [ ]:
# Merge dataframes to create a user-item matrix based on order frequency
user_item_matrix = pd.merge(food_rating, food_data, on="FoodID")

# Create a pivot table for user-item matrix
pivot_table = user_item_matrix.pivot_table(index='UserID', columns='FoodName', aggfunc='size', fill_value=0)

# Calculate cosine similarity between users based on order frequency
user_similarity = cosine_similarity(pivot_table.T)


In [109]:
def recommend_popular_food(user_id, n=3, num_similar_users=5):
    user_index = pivot_table.index.get_loc(user_id)
    similar_users_indices = sorted(range(len(user_similarity[user_index])), key=lambda i: user_similarity[user_index][i], reverse=True)[1:num_similar_users + 1]

    common_items = set()

    for i in similar_users_indices:
        user_data = user_item_matrix[user_item_matrix['UserID'] == i + 1]
        most_ordered_food = user_data.groupby('FoodName').size().sort_values(ascending=False).index.tolist()

        # Filter out items the target user has already ordered
        most_ordered_food = [food for food in most_ordered_food if food not in pivot_table.columns[user_index]]

        # Update the set with unique items from each similar user
        common_items.update(most_ordered_food)

    # Limit to the top n common items
    final_common_items = list(common_items)[:n]
    return final_common_items


In [111]:
# Example: Get recommendations for User
User_ID = int(input("Please enter a UserID to get recommendations"))
recommen = recommend_popular_food(User_ID, n=3)
print("POPULAR FOOD ORS running to fetch results..")

print(f"Recommended foods for {User_ID} : {recommen}")

Top 3 Most Ordered Items from Top 5 Similar Users: ['Baba Ghanoush (80 Gm) (D)', 'JUMBO Non Veg Kebab Meal Combo(0.6 Kg) 2-3 Persons TOPSELLER [DT]', 'Chicken Lahmacun']
POPULAR FOOD ORS running to fetch results..
Recommended foods for 1 : ['Baba Ghanoush (80 Gm) (D)', 'JUMBO Non Veg Kebab Meal Combo(0.6 Kg) 2-3 Persons TOPSELLER [DT]', 'Chicken Lahmacun']
